In [65]:
from SmartApi import SmartConnect #or from SmartApi.smartConnect import SmartConnect
import pyotp
from logzero import logger
import numpy as np
import pandas as pd
from datetime import datetime as dt
from time import sleep
from datetime import datetime,timedelta
from requests.exceptions import ReadTimeout  # Add this import for handling ReadTimeout

def Connect_Server():
    api_key = '.............'
    secret_key='.............'
    username = '.............'
    pwd = '6767'
    smartApi = SmartConnect(api_key)
    try:
        token = '.................'
        totp = pyotp.TOTP(token).now()
    except Exception as e:
        logger.error("Invalid Token: The provided token is not valid.")
        raise e
    
    correlation_id = "abcde"
    data = smartApi.generateSession(username, pwd, totp)
    sleep(0.7)
    
    if data['status'] == False:
        logger.error(data)
        
    else:
        # login api call
        # logger.info(f"You Credentials: {data}")
        authToken = data['data']['jwtToken']
        refreshToken = data['data']['refreshToken']
        # fetch the feedtoken
        feedToken = smartApi.getfeedToken()
        # fetch User Profile
        sleep(0.7)
        res = smartApi.getProfile(refreshToken)
        sleep(0.7)
        smartApi.generateToken(refreshToken)
        sleep(0.7)
        res=res['data']['exchanges']
        print(res)
    return(smartApi)

def token(stock_symbol_name,exchange):
    sleep(0.7)
    srch=smartApi.searchScrip(exchange,stock_symbol_name)['data'][0]
    symbol=srch['tradingsymbol']
    return(srch['symboltoken'])
def Get_15_df(symbol_token,dd,exchange):
    
    formatted_now = (dd+timedelta(days=15)).strftime("%Y-%m-%d")
    
    formatted_last=(dd-timedelta(days=4)).strftime("%Y-%m-%d")
    d=pd.DataFrame()
    
    hp={
        "exchange": exchange,
        "symboltoken": symbol_token,
        "interval": "FIFTEEN_MINUTE",
        "fromdate": formatted_last+" 09:00", 
        "todate": (formatted_now+" 16:00")
        }
    try:
        sleep(0.7)
        a=smartApi.getCandleData(hp)
        d=pd.DataFrame(a['data'])
        d.set_index(0,inplace=True)
        d.index=pd.to_datetime(d.index)
        d.rename(columns={1:"Open",2:"High",3:"Low",4:"Close",5:'Volume'},inplace=True)
        d=d.iloc[-100:]
        d['6ma']=d.Close.rolling(6).mean()
        d['14ma']=d.Close.rolling(14).mean()
        print("15m data fetched successfully",symbol_token,d.index[-1])
    except:
        try:
            print("15 min data fetching issue trying again")
            sleep(1)
            a=smartApi.getCandleData(hp)
            d=pd.DataFrame(a['data'])
            d.set_index(0,inplace=True)
            d.index=pd.to_datetime(d.index)
            d.rename(columns={1:"Open",2:"High",3:"Low",4:"Close",5:'Volume'},inplace=True)
            d=d.iloc[-100:]
            d['6ma']=d.Close.rolling(6).mean()
            d['14ma']=d.Close.rolling(14).mean()
        except:
            print("15 min data not fetched")
            pass
                

    return(d)


def Get_1_df(symbol_token,dd,exchange):
    
    formatted_now = (dd+timedelta(days=15)).strftime("%Y-%m-%d")
    
    formatted_last=(dd-timedelta(days=4)).strftime("%Y-%m-%d")
    df=pd.DataFrame()
    
    hp2={
        "exchange": exchange,
        "symboltoken": symbol_token,
        "interval": "ONE_MINUTE",
        "fromdate": formatted_last+" 09:00", 
        "todate": (formatted_now+" 16:00")
        }
    try:
        sleep(1)
        df=pd.DataFrame(smartApi.getCandleData(hp2)['data']).iloc[-10:]
        df.set_index(0,inplace=True)
        df.index=pd.to_datetime(df.index)
        df.rename(columns={1:"Open",2:"High",3:"Low",4:"Close",5:'Volume'},inplace=True)
        print("1m data fetched successfully",symbol_token,df.index[-1])
        
    except:
        try:
            print("1 min data fetching issue trying again")
            sleep(1)
            df=pd.DataFrame(smartApi.getCandleData(hp2)['data']).iloc[-10:]
            df.set_index(0,inplace=True)
            df.index=pd.to_datetime(df.index)
            df.rename(columns={1:"Open",2:"High",3:"Low",4:"Close",5:'Volume'},inplace=True)
        except:
            print("1 min data not fetched")
            pass
        
        

    return(df)


def place_order(symbol,symbol_token,Quantity,exchange,Order_type):
    orderparams = {
            "variety": "NORMAL",
            "tradingsymbol": symbol,
            "symboltoken": symbol_token,
            "transactiontype": Order_type,
            "exchange": exchange,
            "ordertype": "MARKET",
            "producttype": "INTRADAY",
            "duration": "DAY",
            "price": "0",
            "squareoff": "0",
            "stoploss": "0",
            "quantity": Quantity
            }
    sleep(0.7)
    smartApi.placeOrder(orderparams)
    sleep(0.7)
    average_price=float(smartApi.orderBook()['data'][-1]['averageprice'])
    return(average_price)

def Order_book(pos,symbol):
    c=0
    sleep(0.7)
    order_book=smartApi.orderBook()
    buy_price=0
    if pos['data']!=None:
        
        for j in order_book['data']:
            if (j['status']=='complete') and (j['tradingsymbol']==symbol):
                
                if (j['transactiontype']=='BUY'):
                    buy_price=float(j['averageprice'])
                    c+=1
                elif (j['transactiontype']=='SELL'):
                    buy_price=0
                    
            elif (j['tradingsymbol']==symbol):
                buy_price=0
        print("buy price ",buy_price)
    return(buy_price,c)  

def Position(buy_price,symbol_token,pos):
    net_qnt=0
    pl=0
    unrealized=0
    if pos['data']!=None:
        
        for i in pos['data']:
            if(i['symboltoken']==symbol_token):

                net_qnt=int(i['netqty'])

                if net_qnt!=0:#smartApi.orderBook()['data'][-1]['transactiontype']=="BUY":
                    print("ltp is:",i['ltp'])
                    # buy_price=float(smartApi.orderBook()['data'][-1]['averageprice'])
                    pl=round(((float(i['ltp'])-float(buy_price))*100/(float(buy_price))),3)
                    unrealized=(i['unrealised'])#pl*net_qnt*buy_price
    return(net_qnt,pl,unrealized)





smartApi=Connect_Server()




def Start_Trading(item={"name":"nifty26jun","suf":"CE","list_name":pd.DataFrame(),"SL_pct":-4,"TP_pct":400,"update_sl_tp":True,"exchange":"NFO" ,"Quantity":1 },print_data=True,wait_sleep=0.5,smartApi=Connect_Server()):
    
    print("trading started")
    pos=smartApi.position()
    dd = (datetime.now())
    item['max_pl']=0 
    item['pl']=0
    item['last']=0
    item['net_qnt']=0
    item['count']=0
    print("item printing")
    item['open_position']=False
    Stop_Trad=False
    list_updated=False
   

    item['SL']=item['SL_pct']
    item['TP']=item['TP_pct']  

    lis=smartApi.searchScrip(item["exchange"],item["name"])['data']
    dic={"name":[],"token":[],"ltp":[],'diff':[]}

    for ticker in lis:
        
        try:
            if ticker['tradingsymbol'].endswith(item['suf']):
                ltp=smartApi.ltpData(item['exchange'], ticker['tradingsymbol'], ticker['symboltoken'])['data']['ltp']
                dic['ltp'].append(ltp)
                dic['diff'].append(abs(ltp-110))
                dic['name'].append(ticker['tradingsymbol'])
                dic['token'].append(ticker['symboltoken'])
                
        except:
            pass
    df=pd.DataFrame(dic).sort_values('diff')
    item["symbol"]=df.name[0]
    item['symbol_token']=df.token[0]
    print("selected sym:",item["symbol"],item['symbol_token'])

    item['list_name']=df
    (item['buy_price'],item['count'])=Order_book(pos,item['symbol'])
    (item['net_qnt'],item['pl'],item['unrealized'])=Position(buy_price=item['buy_price'],symbol_token=item['symbol_token'],pos=pos)
    if item['net_qnt']!=0:
        item['open_position']=True
    print("item",item)
    sleep(0.7)
    pos=smartApi.position() 
    if print_data:
        print("####"*30)
        print(f"{dd} Symbol: {item['symbol']} symbol_token: {item['symbol_token']} QNT: {item['Quantity']} SL: {item['SL']} TP: {item['TP']}") 
 

#     try: 
    print("*"*50)
    while True:
        dd = (datetime.now())
                
        if (dd.strftime('%H:%M')=="09:10"):
            Stop_Trad=False
            list_updated=False

            sleep(0.5) 
            pos=smartApi.position()
            sleep(0.3)
            dd = (datetime.now())
            item['max_pl']=0
            item['pl']=0
            item['last']=0
            item['net_qnt']=0
            item['count']=0
        
            item['SL']=item['SL_pct']
            item['TP']=item['TP_pct'] 
            item['open_position']=False
            
            (item['buy_price'],item['count'])=Order_book(pos,item['symbol'])  
             
            (item['net_qnt'],item['pl'],item['unrealized'])=Position(buy_price=item['buy_price'],symbol_token=item['symbol_token'],pos=pos)

            if item['net_qnt']!=0:
                item['open_position']=True
                
            lis=smartApi.searchScrip(item["exchange"],item["name"])['data']
            dic={"name":[],"token":[],"ltp":[],'diff':[]}
        
            for ticker in lis:
        
                try:
                    if ticker['tradingsymbol'].endswith(item.suf):
                        ltp=smartApi.ltpData(item['exchange'], ticker['tradingsymbol'], ticker['symboltoken'])['data']['ltp']
                        dic['ltp'].append(ltp)
                        dic['diff'].append(abs(ltp-110))
                        dic['name'].append(ticker['tradingsymbol'])
                        dic['token'].append(ticker['symboltoken'])
                        
                except:
                    pass
            df=pd.DataFrame(dic).sort_values('diff')
            item["symbol"]=df.name[0]
            item['symbol_token']=df.token[0]
            ptint("selected sym:",item["symbol"])

            item['list_name']=df
            item["symbol"]=df.name[0]
            item['list_name']=df
            print("selected sym:",item["symbol"],item['symbol_token'])


            
            
            
            if print_data:
                print(f"{dd} Symbol: {item['symbol']} symbol_token: {item['symbol_token']} QNT: {item['Quantity']} SL: {item['SL']} TP: {item['TP']} ")

        if ((dd.strftime('%H:%M')>="09:15")and (dd.strftime('%H:%M')<="15:30") and (item['count']<7) and ((dd.isoweekday()!=6) and  (dd.isoweekday()!=7))):
            (item['net_qnt'],item['pl'],item['unrealized'])=Position(buy_price=item['buy_price'],symbol_token=item['symbol_token'],pos=pos)

            sleep(0.6)
            item['new']=smartApi.ltpData(item['exchange'], item['symbol'], item['symbol_token'])['data']['ltp']

            pos=smartApi.position()

            if (dd.strftime('%H:%M') == "09:15") and (list_updated==False):
                dic = {"name": [], "token": [], "ltp": [], 'diff': []}

                for _, ticker in item["list_name"].head(30).iterrows():
                    try:
                        if str(ticker.name).endswith(item['suf']):
                            print(ticker.name)
                
                            
                            
                            ltp = smartApi.ltpData(item['exchange'], ticker.name, ticker.token)['data']['ltp']
                            dic['ltp'].append(ltp)
                            dic['diff'].append(abs(ltp - 110))
                            dic['name'].append(ticker.name)
                            dic['token'].append(ticker.token)
                            list_updated=True
                    except Exception as e:
                        print("Error fetching LTP:", e)

                df=pd.DataFrame(dic).sort_values('diff')
                
                item["symbol"]=df.name[0]
                item['symbol_token']=df.token[0]
                print("selected sym:",item["symbol"],item['symbol_token'])
                
            
            if ((item['net_qnt']==0)) and dd.strftime('%H:%M') == "9:15" and 27 <= dd.second <= 32:#
                print("order Buy(set time matched)")
                item['buy_price']=place_order(symbol=item['symbol'],
                                    symbol_token=item['symbol_token'],
                                    Quantity=item['Quantity'],
                                    exchange=item['exchange'],
                                    Order_type="BUY")
                item['open_position']=True

                item['SL']=item['SL_pct']
                item['TP']=item['TP_pct']
                item['max_pl']=0
                item['count']=item['count']+1
                pos=smartApi.position()

                sleep(0.4)
                (item['net_qnt'],item['pl'],item['unrealized'])=Position(buy_price=item['buy_price'],symbol_token=item['symbol_token'],pos=pos)

            
            sleep(0.4) 
            sleep(wait_sleep)
            dd = (datetime.now()) 
            

         
           
            item['new']=smartApi.ltpData(item['exchange'], item['symbol'], item['symbol_token'])['data']['ltp']

                
            if item['new']!=item['last']:
                item['last']=item['new']
                if print_data:
                    print("####"*40)
                    print(
                            dd, item['last'], item['symbol'],
                            "total_trades:", item['count'],
                            "ltp",item['new'] ,"current PL:",item['pl'],
                            f"max_pl: {item['max_pl']} sl: {item['SL']} tp: {item['TP']}"
                        )

                               
                if (item['net_qnt']==0) and (int(item['buy_price'])==int(item['new'])) and Stop_Trad==False:
                    item['buy_price']=place_order(symbol=item['symbol'],
                                        symbol_token=item['symbol_token'],
                                        Quantity=item['Quantity'],
                                        exchange=item['exchange'],
                                        Order_type="BUY")
                    item['open_position']=True

                    
                    item['SL']=item['SL_pct']
                    item['TP']=item['TP_pct']
                    item['max_pl']=0
                    item['count']=item['count']+1
                    sleep(0.4)
                    pos=smartApi.position()
                    (item['buy_price'],_)=Order_book(pos,item['symbol'])
                    (item['net_qnt'],item['pl'],item['unrealized'])=Position(buy_price=item['buy_price'],symbol_token=item['symbol_token'],pos=pos)


                  
                        
                    if print_data:
                        print(dd,' ',item['symbol'], " Purchased: ",item['buy_price'],"is_open_pos",item['open_position'])
            
                if (item['net_qnt']>0) :
     
                    
                    if print_data:
                        print(item['symbol'],"net qnt",item['net_qnt'],"p&l:",item['pl'],"%","Unrealized Value(Rs.): ", item['unrealized'] )
    
                    if (item['max_pl']<item['pl']) and (item['update_sl_tp']) :#(pl>TP) and (update_sl_tp) :
                        item['max_pl']=item['pl']
                        if (item['SL']!=(int(item['max_pl'])-1)+(1+item['SL'])):#-0.9 !=-1 (0.5-1)+(1-0.9)
                            item['TP']=int(item['max_pl'])+3
                            item['SL']=item['SL']+(int(item['max_pl']))
                            
                        if print_data:
                            print(f"{dd} {item['symbol']} Stop_loss % and Target_price % Changed new sl {item['SL']} new TP {item['TP']}")
                        
                    if (item['pl']<item['SL']):
                
                        place_order(symbol=item['symbol'],
                            symbol_token=item['symbol_token'],
                            Quantity=item['Quantity'],
                            exchange=item['exchange'],
                            Order_type="SELL")
                        print("position closed pl<sl")
                        item['open_position']=False
                        sleep(0.7)
                        pos=smartApi.position()
                        (item['net_qnt'],item['pl'],item['unrealized'])=Position(buy_price=item['buy_price'],symbol_token=item['symbol_token'],pos=pos)
    
                        if print_data:
                            print(dd,' pl<sl ',item['symbol']," Position Closed :",item['net_qnt'],"p&l :",round(item['pl'],2),"%")
                            
                    elif (item['pl']>item["TP"]):
                
                        place_order(symbol=item['symbol'],
                            symbol_token=item['symbol_token'],
                            Quantity=item['Quantity'],
                            exchange=item['exchange'],
                            Order_type="SELL")
                        Stop_Trad=True
                        print("position closed pl>tp")
                        item['open_position']=False
                        sleep(0.7)
                        pos=smartApi.position()
                        (item['net_qnt'],item['pl'],item['unrealized'])=Position(buy_price=item['buy_price'],symbol_token=item['symbol_token'],pos=pos)
    
                        if print_data:
                            print(dd,' pl>tp ',item['symbol']," Position Closed :",item['net_qnt'],"p&l :",round(item['pl'],2),"%")
                            
                    elif (item['pl']<60) and (item['max_pl']>80):
                
                        place_order(symbol=item['symbol'],
                            symbol_token=item['symbol_token'],
                            Quantity=item['Quantity'],
                            exchange=item['exchange'],
                            Order_type="SELL")
                        print("position closed 60,80")
                        item['open_position']=False
                        sleep(0.7)
                        pos=smartApi.position()
                        (item['net_qnt'],item['pl'],item['unrealized'])=Position(buy_price=item['buy_price'],symbol_token=item['symbol_token'],pos=pos)
    
                        if print_data:
                            print(dd,' pl<60, and max pl>80 ',item['symbol']," Position Closed :",item['net_qnt'],"p&l :",round(item['pl'],2),"%")

[I 250620 15:09:49 smartConnect:121] in pool


['nse_fo', 'nse_cm', 'cde_fo', 'ncx_fo', 'bse_fo', 'bse_cm', 'mcx_fo']


[I 250620 15:09:54 smartConnect:121] in pool


['nse_fo', 'nse_cm', 'cde_fo', 'ncx_fo', 'bse_fo', 'bse_cm', 'mcx_fo']


In [67]:
Start_Trading(item={"name":"nhpc-eq","list_name":pd.DataFrame(),"suf":"EQ",
                    "SL_pct":-0.02,"TP_pct":0.02,"update_sl_tp":True,"exchange":"NSE" ,"Quantity":1 },print_data=True,wait_sleep=0.5,smartApi=Connect_Server())
    

[I 250620 15:14:32 smartConnect:121] in pool


['nse_fo', 'nse_cm', 'cde_fo', 'ncx_fo', 'bse_fo', 'bse_cm', 'mcx_fo']
trading started
item printing


[I 250620 15:14:36 smartConnect:485] Search successful. Found 1 trading symbols for the given query:
    1. exchange: NSE, tradingsymbol: NHPC-EQ, symboltoken: 17400


selected sym: NHPC-EQ 17400
buy price  82.11
ltp is: 82.08
item {'name': 'nhpc-eq', 'list_name':       name  token    ltp   diff
0  NHPC-EQ  17400  82.08  27.92, 'suf': 'EQ', 'SL_pct': -0.02, 'TP_pct': 0.02, 'update_sl_tp': True, 'exchange': 'NSE', 'Quantity': 1, 'max_pl': 0, 'pl': -0.037, 'last': 0, 'net_qnt': 1, 'count': 1, 'open_position': True, 'SL': -0.02, 'TP': 0.02, 'symbol': 'NHPC-EQ', 'symbol_token': '17400', 'buy_price': 82.11, 'unrealized': '-0.03'}
########################################################################################################################
2025-06-20 15:14:36.543740 Symbol: NHPC-EQ symbol_token: 17400 QNT: 1 SL: -0.02 TP: 0.02
**************************************************
ltp is: 82.08
################################################################################################################################################################
2025-06-20 15:14:41.053058 82.09 NHPC-EQ total_trades: 1 ltp 82.09 current PL: -0.037 max_pl: 0 sl:

KeyboardInterrupt: 